In [13]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [14]:
PlTrackSim = load_npz("target_pl_x_target_tracks_similarity.npz")

In [15]:
PlwithArtist = load_npz("targetPlaylistArtistReduced.npz")

In [16]:
tTracksWithArtist = load_npz("targetTracksArtistReduced.npz")

In [17]:
PlwithArtist = normalize(PlwithArtist, norm="max")

In [18]:
PlTrackSimByArtist = PlwithArtist * tTracksWithArtist.T

In [27]:
artistsTags = load_npz("artist_with_tag_bincount_reduced.npz")
artistsTags = normalize(artistsTags)

In [28]:
P=load_npz("playlist_reduced.npz")
P=normalize(P)

In [29]:
PlArtistSim = P * artistsTags.T

In [32]:
PlTrackSimByArtist = PlArtistSim * tTracksWithArtist.T

In [34]:
finalPlSim = PlTrackSimByArtist + PlTrackSim 

In [35]:
finalPlSim = finalPlSim + PlTrackSimByArtist

In [ ]:
finalPlSim = PlTrackSim.multiply(PlTrackSimByArtist) 

In [ ]:
finalPlSim

In [36]:
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
playlists_with_tracks=load("playlists_with_tracks.npy")
target_tracks_ordered = load("targetTracksOrdered.npy")
def getsimil(pls, similrow):
    maxi = flip(argsort(similrow), axis=0)
    r = []
    for m in maxi:
        if(not isin(target_tracks_ordered[m], pls[1:])):
           r.append(target_tracks_ordered[m])
           if(len(r)==5):
               return r

In [37]:
fname = "tag_artist2_average_similarity.csv"
open(fname,"w").close()
print("playlist_id,track_ids",file=open(fname,"a"))
for pl, simil in zip(playlists_with_tracks, finalPlSim):
    s = str(int(pl[0]))
    s += ","
    r = getsimil(pl, ravel(simil.todense()))
    for el in r:
        s+=str(el)
        s+=" "
    print(s,file=open(fname,"a"))

In [27]:
normalize([[10, 5, 1], [0, 10, 0]], norm="max")

array([[ 1. ,  0.5,  0.1],
       [ 0. ,  1. ,  0. ]])